<a href="https://colab.research.google.com/github/richars7/Automatic_Grading_System/blob/master/Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Character-Level LSTM in PyTorch

In [0]:

# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

    100% |████████████████████████████████| 91.1MB 79.8MB/s 
    100% |████████████████████████████████| 2.0MB 11.1MB/s 
fastai 1.0.42 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.


In [0]:
!pip install PyPDF2

    100% |████████████████████████████████| 81kB 3.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [0]:
import PyPDF2

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  with open(fn, 'r') as file:
     content = file.read()
  

Saving computeCost.txt to computeCost.txt
User uploaded file "computeCost.txt" with length 2512 bytes


In [0]:
import re
clean_text = re.sub(r'%(.*)\n','',content)
clean_text

"function J = computeCost(X, y, theta)\n\nm = length(X); J = 0;\nJ=1/(2*m)*(sum(((X*theta)-y).^2));\nend\n\n\nfunction plotData(x, y)\n\nfigure; \nplot(x,y,'rx','MarkerSize',10);\nxlabel('x-values');\nylabel('y-values');\nend\n\nfunction [theta, J_history] = gradientDescent(X, y, theta, alpha, num_iters)\n\nm = length(X); J_history = zeros(num_iters, 1);\n\nfor iter = 1:num_iters\n                            dell=(1/m)*((X*theta-y)' * X)';\n    theta = theta - (alpha * dell);  \n            J_history(iter) = computeCost(X, y, theta);\nend\n\nend"

In [0]:
!pip install utils

In [0]:
!pip install nltk

In [0]:
import utils
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
words = nltk.word_tokenize(clean_text)
words
   

['function',
 'J',
 '=',
 'computeCost',
 '(',
 'X',
 ',',
 'y',
 ',',
 'theta',
 ')',
 'm',
 '=',
 'length',
 '(',
 'X',
 ')',
 ';',
 'J',
 '=',
 '0',
 ';',
 'J=1/',
 '(',
 '2*m',
 ')',
 '*',
 '(',
 'sum',
 '(',
 '(',
 '(',
 'X*theta',
 ')',
 '-y',
 ')',
 '.^2',
 ')',
 ')',
 ';',
 'end',
 'function',
 'plotData',
 '(',
 'x',
 ',',
 'y',
 ')',
 'figure',
 ';',
 'plot',
 '(',
 'x',
 ',',
 'y',
 ',',
 "'rx",
 "'",
 ',',
 "'MarkerSize',10",
 ')',
 ';',
 'xlabel',
 '(',
 "'x-values",
 "'",
 ')',
 ';',
 'ylabel',
 '(',
 "'y-values",
 "'",
 ')',
 ';',
 'end',
 'function',
 '[',
 'theta',
 ',',
 'J_history',
 ']',
 '=',
 'gradientDescent',
 '(',
 'X',
 ',',
 'y',
 ',',
 'theta',
 ',',
 'alpha',
 ',',
 'num_iters',
 ')',
 'm',
 '=',
 'length',
 '(',
 'X',
 ')',
 ';',
 'J_history',
 '=',
 'zeros',
 '(',
 'num_iters',
 ',',
 '1',
 ')',
 ';',
 'for',
 'iter',
 '=',
 '1',
 ':',
 'num_iters',
 'dell=',
 '(',
 '1/m',
 ')',
 '*',
 '(',
 '(',
 'X*theta-y',
 ')',
 "'",
 '*',
 'X',
 ')',
 "'",
 ';',
 't

In [0]:
# print some stats about this word data
print("Total words in text: {}".format(len(words)))
print("Unique words: {}".format(len(set(words)))) # `set` removes any duplicate words

Total words in text: 157
Unique words: 49


In [0]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

print(int_words[:30])

AttributeError: ignored

In [0]:
char2int = {ch: ii for ii, ch in int2char.items()}

In [0]:
char2int

{'\n': 46,
 ' ': 10,
 '%': 23,
 '(': 40,
 ')': 20,
 '*': 4,
 ',': 48,
 '-': 0,
 '.': 28,
 '/': 35,
 '0': 17,
 '1': 25,
 '2': 37,
 ':': 1,
 ';': 53,
 '=': 2,
 'C': 26,
 'D': 7,
 'E': 32,
 'H': 13,
 'I': 19,
 'J': 27,
 'M': 43,
 'O': 3,
 'P': 15,
 'R': 24,
 'S': 42,
 'T': 31,
 'U': 36,
 'X': 18,
 'Y': 12,
 '^': 47,
 'a': 38,
 'b': 50,
 'c': 11,
 'd': 44,
 'e': 5,
 'f': 41,
 'g': 9,
 'h': 30,
 'i': 21,
 'l': 34,
 'm': 45,
 'n': 14,
 'o': 33,
 'p': 49,
 'r': 22,
 's': 52,
 't': 29,
 'u': 8,
 'v': 6,
 'x': 16,
 'y': 51,
 'z': 39}

In [0]:
import numpy as np
encoded = np.array([char2int[ch] for ch in content])

In [0]:

encoded[:100]

array([41,  8, 14, 11, 29, 21, 33, 14, 10, 27, 10,  2, 10, 11, 33, 45, 49,
        8, 29,  5, 26, 33, 52, 29, 40, 18, 48, 10, 51, 48, 10, 29, 30,  5,
       29, 38, 20, 46, 23, 26,  3, 43, 15, 36, 31, 32, 26,  3, 42, 31, 10,
       26, 33, 45, 49,  8, 29,  5, 10, 11, 33, 52, 29, 10, 41, 33, 22, 10,
       34, 21, 14,  5, 38, 22, 10, 22,  5,  9, 22,  5, 52, 52, 21, 33, 14,
       46, 23, 10, 10, 10, 27, 10,  2, 10, 26,  3, 43, 15, 36, 31])

Pre-Processing the data 

In [0]:
def one_hot_encode(arr, n_labels):
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [0]:
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [0]:
test_seq

array([[3, 5, 1]])

Make training mini batches

In [0]:

def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [0]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [0]:

# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[41  8 14 11 29 21 33 14 10 27]
 [10 26 33 45 49  8 29  5 10 11]
 [32 26  3 42 31 40 18 48 10 51]
 [38 10 38 52 10 29 30  5 46 23]
 [29 10 29 30  5 10 44 38 29 38]
 [ 5 41  8 34 10  6 38 34  8  5]
 [38 45 49 34  5 52 46 23 10  2]
 [ 2  2  2  2  2  2  2  2  2  2]]

y
 [[ 8 14 11 29 21 33 14 10 27 10]
 [26 33 45 49  8 29  5 10 11 33]
 [26  3 42 31 40 18 48 10 51 48]
 [10 38 52 10 29 30  5 46 23 10]
 [10 29 30  5 10 44 38 29 38 10]
 [41  8 34 10  6 38 34  8  5 52]
 [45 49 34  5 52 46 23 10  2  2]
 [ 2  2  2  2  2  2  2  2  2  2]]


In [0]:
import torch
from torch import nn
import torch.nn.functional as F
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [0]:
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(54, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=54, bias=True)
)


In [0]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

In [0]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
       
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [0]:
def sample(net, size, prime='The', top_k=None):

    net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [0]:
print(sample(net, 1000, prime='function', top_k=5))

function;)..);;Ee..eE)E;;e)Ee.eeeE.;EE)e.;e;.EE;Ee;eE;).E)eEe);eEe.e.);;;e.E;;eee;;E.e.eE.e.e.e)EE.)e))e;.).e;EEEE).EEeEE)e;.)))e);EEe;EE));;EE;).Ee);.ee).);eEE;e))e));)EE)eE;Ee.)E.;;.;.)e;.)))e;;e)e.e;EE....))EE).)..E..eE.e;.eEee;Ee)E)E;).E.eeEEEe;.e;;e..E.eeE)eE)EEeE;eE;EEe)));E;)e;.EeeeE.;;.;EeE;E);E)e.Ee.;e);.);.;)eee)e..))..).)e.)EE;e).;EEEe))E).);.Ee).)E).)Ee;..e;...;..e)eEEE.;.E)eeE..;)e.;EeE))e.ee;E)E;eEe)););;EeEE);;e;eee.)..eE);e;.;;Ee);E.)Ee;ee)););.);)e;;)..e;ee;;.E)E.)eE.);)EE).)E..eEe)E.e.e))Eee;;))eEE;E;eE)EeE).eE.)E)eee.E;.e);.e);..;)e;eE);ee.;).E;Ee)E;E..E).;).eeeee)))EeE)Ee...;.e;.Eee)e)EeE.)EE;E;e..EeE;EE;E;;;.E..eE);Ee.;;eee)E..)..EeE)).)E.....E;.Ee..).)E;E.)e))));E;E);.e.)e;EEE))E.)e.;Eee;;).;E;.E;)e;.;.eeE..;EE...;.)....)))e...e)..E;;.e);..E);ee.e.;.e.eE);.E...eeeeEe;ee;;.Ee.;;E;.Ee;)E.EeE.e.E))EE;).);;e.E.E;);e.e);...);))).e..EE)E)EEE.).;e.E);).;)e)E.;;EEe).)E.E);.e)e.E.EE;..eE))e;;;E).;e)eE)e.E.ee;.ee.;.))...;..e.e)e);ee..E.E);Ee.eEEee)eEE;e.;e)e.eE.EE)Ee.e.).E)